In [1]:
# https://www.twilio.com/blog/2017/02/an-easy-way-to-read-and-write-to-a-google-spreadsheet-in-python.html
# !pip install gspread oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_formatting import *
import pandas as pd
import numpy as np
import time
import geopandas as gpd
import datetime as dt

In [2]:
first_date = '2020-08-01'
last_date = '2020-12-16'

weekday_mapping = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}

alldates = pd.date_range(start='2020-01-01', end='2020-12-31').tolist()
datelist = [str(i.date()) for i in alldates]
weekdays = [weekday_mapping[i.date().weekday()] for i in alldates]
weeklist = [i//7 for i in range(len(alldates))]
DATE_DF = pd.DataFrame({'date':datelist,'day':weekdays,'week':weeklist})
DATE_DF = DATE_DF.loc[(DATE_DF['date']<=last_date)&(DATE_DF['date']>=first_date)].copy()
DATERANGE = DATE_DF['date'].tolist()

date_dict = {DATERANGE[i]:i for i in range(len(DATERANGE))}

In [4]:
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds']
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('../../universities/data/client_secret.json', scope)
client = gspread.authorize(creds)

In [5]:
# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
referen_sheet = client.open("00_Reference")
summary_sheet = client.open("00_Summary")
summary_data_sheet = summary_sheet.worksheet('summary')

In [6]:
ref_data = pd.DataFrame(referen_sheet.worksheet('reference'
                                    ).get_all_records()).iloc[:,:-1]
completed_df = ref_data.loc[ref_data['status'].isin(['completed'])].copy()
all_data = ref_data.loc[ref_data['status'].isin(['completed','cumulative only'])].copy()
to_add = all_data.copy()

# Watch out, the next step takes several hours to pull
(there's almost certainly faster ways to do this, but I add sleep steps in between each new sheet)

In [7]:
add_dfs = []
nope_ipeds = []
for st in ['Indiana']: # as an example, just look at indiana
# for st in to_add.state.unique():
    state_sheet = client.open(st)
    ipdfs = to_add.loc[to_add['state']==st]['ipeds_id'].unique()
    print(st,"-",len(ipdfs),"schools left",time.strftime("%H:%M:%S", time.localtime()))
    for ip in ipdfs:
        try:
            print("\t",to_add.loc[to_add['ipeds_id']==ip].college.values[0],
                  time.strftime("%H:%M:%S", time.localtime()))
            add_dfs.append(pd.DataFrame(state_sheet.worksheet(str(ip)).get_all_records()))
            time.sleep(15)
        except:
            print('\t\tcould not do',ip)
            nope_ipeds.append(ip)

    time.sleep(30)

Indiana - 18 schools left 12:06:54
	 Indiana University-East 12:06:54
	 Indiana University-Kokomo 12:07:10
	 Indiana University-South Bend 12:07:26
	 Indiana University-Southeast 12:07:41
	 Anderson University 12:07:57
	 Butler University 12:08:12
	 Holy Cross College 12:08:28
	 Purdue University Fort Wayne 12:08:44
	 Ball State University 12:08:59
	 Franklin College 12:09:15
	 Indiana University-Bloomington 12:09:31
	 Indiana University-Purdue University-Indianapolis 12:09:46
	 Ivy Tech Community College 12:10:02
	 Purdue University-Main Campus 12:10:18
	 University of Evansville 12:10:33
	 University of Indianapolis 12:10:49
	 University of Notre Dame 12:11:05
	 University of Southern Indiana 12:11:21


In [11]:
curr_df = pd.concat(add_dfs).reset_index().iloc[:,1:]

curr_df.loc[curr_df['positive_tests']==' ','positive_tests'] = np.nan
curr_df['ipeds_id'] = curr_df['ipeds_id'].astype(str)
curr_df = curr_df.loc[curr_df['date']<=last_date]

curr_df['total_tests'] = curr_df['total_tests'].astype(str)
curr_df['positive_tests'] = curr_df['positive_tests'].astype(str)

curr_df['total_tests'] = curr_df['total_tests'].str.replace(',','')
curr_df['positive_tests'] = curr_df['positive_tests'].str.replace(',','')

curr_df.loc[curr_df['total_tests']=='','total_tests'] = np.nan
curr_df.loc[curr_df['positive_tests']=='','positive_tests'] = np.nan

curr_df['total_tests'] = curr_df['total_tests'].astype(float)
curr_df['positive_tests'] = curr_df['positive_tests'].astype(float)
curr_df['total_tests'] = np.nan_to_num(curr_df['total_tests'])
curr_df['positive_tests'] = np.nan_to_num(curr_df['positive_tests'])

curr_df = curr_df[['date', 'total_tests', 'positive_tests', 'college', 'url', 'ipeds_id', 'notes']]
curr_df = curr_df.merge(DATE_DF,how='left',on='date')
curr_df['state'] = 'Indiana'
curr_df.loc[curr_df['notes']=='','notes'] = np.nan

curr_df = curr_df.loc[curr_df['date']>'2020-07-30'].copy()

print(curr_df['college'].nunique())
print(curr_df.shape)
curr_df.head()

18
(2484, 10)


,date,total_tests,positive_tests,college,url,ipeds_id,notes,day,week,state
0,2020-08-01,0.0,0.0,Indiana University-East,https://www.iue.edu/covid/,151388,NaN,Saturday,30,Indiana
1,2020-08-02,0.0,0.0,Indiana University-East,https://www.iue.edu/covid/,151388,NaN,Sunday,30,Indiana
2,2020-08-03,0.0,0.0,Indiana University-East,https://www.iue.edu/covid/,151388,NaN,Monday,30,Indiana
3,2020-08-04,0.0,0.0,Indiana University-East,https://www.iue.edu/covid/,151388,NaN,Tuesday,30,Indiana
4,2020-08-05,0.0,0.0,Indiana University-East,https://www.iue.edu/covid/,151388,NaN,Wednesday,31,Indiana


In [12]:
# curr_df.to_csv('../data/ind_df.csv',index=False)